In [8]:
import pandas as pd
import numpy as np
import chardet
import io

def diagnostic_test(file_path):
    print(f"--- [1. 파일 로딩 시작: {file_path}] ---")
    
    # 인코딩 확인
    with open(file_path, 'rb') as f:
        raw = f.read()
        encoding = chardet.detect(raw)['encoding']
    print(f"인코딩 감지 결과: {encoding}")

    try:
        # 파일 읽기
        df = pd.read_csv(file_path, encoding=encoding, low_memory=False)
        # 모든 컬럼을 일단 문자열로 변환 (숫자형 헤더 방지)
        df.columns = [str(c) for c in df.columns]
        print(f"전체 데이터 모양: {df.shape}")
    except Exception as e:
        print(f"파일 로드 실패: {e}")
        return

    # LITE(RH) 기준 설정
    config = {"name": "Analog Mic", "type": "analog", "range": range(6, 47)}
    cols = df.columns[config["range"]]
    
    print(f"\n--- [2. 컬럼 및 규격 확인] ---")
    print(f"분석 범위 컬럼 샘플: {list(cols[:5])} ... {list(cols[-5:])}")
    
    # 1kHz 위치 찾기
    freqs = [float(str(c).split('.')[0]) for c in cols]
    idx_1k = np.argmin(np.abs(np.array(freqs) - 1000))
    col_1k = cols[idx_1k]
    print(f"1kHz로 판정된 컬럼명: '{col_1k}' (타입: {type(col_1k)})")
    
    limit_low_1k = df.iloc[0][col_1k]
    limit_high_1k = df.iloc[1][col_1k]
    print(f"1kHz 규격 값: Low({limit_low_1k}), High({limit_high_1k})")

    # 데이터 필터링
    sn_col = df.columns[3]
    test_data = df.iloc[2:].dropna(subset=[sn_col])
    test_data = test_data[test_data[sn_col].astype(str).str.contains('/', na=False)].reset_index(drop=True)
    print(f"유효 시료 수: {len(test_data)} EA")

    print(f"\n--- [3. 시료별 데이터 추출 테스트 (상위 5개)] ---")
    extracted_vals = []
    for idx, row in test_data.iterrows():
        raw_val = row[col_1k]
        num_val = pd.to_numeric(raw_val, errors='coerce')
        extracted_vals.append(num_val)
        
        if idx < 5:
            print(f"시료[{idx}] SN: {row[sn_col][:15]}... | 원본값: {raw_val} | 변환값: {num_val}")

    print(f"\n--- [4. 통계 계산 시뮬레이션] ---")
    v = np.array(extracted_vals)
    v_clean = v[~np.isnan(v)]
    
    print(f"추출된 전체 값 개수: {len(extracted_vals)}")
    print(f"NaN이 아닌 정상 숫자 개수: {len(v_clean)}")
    
    if len(v_clean) > 0:
        print(f"최종 결과 -> Min: {v_clean.min():.4f}, Max: {v_clean.max():.4f}, Avg: {v_clean.mean():.4f}")
    else:
        print("결과: 모든 값이 NaN입니다! 추출 과정에서 숫자로 변환되지 못했습니다.")
        
    print(f"\n--- [5. 데이터 타입 정밀 점검] ---")
    sample_raw = test_data[col_1k].iloc[0]
    print(f"데이터 원본 타입: {type(sample_raw)}")
    if isinstance(sample_raw, str):
        print(f"문자열 내 특수문자 포함 여부(ASCII): {[ord(c) for c in sample_raw]}")

# 실행
diagnostic_test('ok.csv')

--- [1. 파일 로딩 시작: ok.csv] ---
인코딩 감지 결과: GB2312
전체 데이터 모양: (275, 105)

--- [2. 컬럼 및 규격 확인] ---
분석 범위 컬럼 샘플: ['100.000', '112.000', '125.000', '140.000', '160.000'] ... ['6300.000', '7100.000', '8000.000', '9000.000', '10000.000']
1kHz로 판정된 컬럼명: '1000.000' (타입: <class 'str'>)
1kHz 규격 값: Low(-11.0), High(-9.0)
유효 시료 수: 260 EA

--- [3. 시료별 데이터 추출 테스트 (상위 5개)] ---
시료[0] SN: 96575GJ200/6A26... | 원본값: -10.419 | 변환값: -10.419
시료[1] SN: 96575GJ200/6A26... | 원본값: -10.397 | 변환값: -10.397
시료[2] SN: 96575GJ200/6A26... | 원본값: -10.365 | 변환값: -10.365
시료[3] SN: 96575GJ200/6A26... | 원본값: -10.512 | 변환값: -10.512
시료[4] SN: 96575GJ200/6A26... | 원본값: -10.308 | 변환값: -10.308

--- [4. 통계 계산 시뮬레이션] ---
추출된 전체 값 개수: 260
NaN이 아닌 정상 숫자 개수: 260
최종 결과 -> Min: -10.9190, Max: -9.7870, Avg: -10.3680

--- [5. 데이터 타입 정밀 점검] ---
데이터 원본 타입: <class 'numpy.float64'>


In [19]:
import xlsxwriter

def generate_recorded_layout():
    # 1. 파일 생성
    output = "Dashboard_Excel.xlsx"
    workbook = xlsxwriter.Workbook(output)
    ws = workbook.add_worksheet('📊 레이아웃 가이드')

    # [Fix 1] 시스템 속성 충돌 방지를 위한 딕셔너리 기반 서식 정의
    base_blue = {'bold': True, 'bg_color': '#DEEAF6', 'align': 'center', 'valign': 'vcenter', 'border': 1}
    base_green = {'bold': True, 'bg_color': '#E2EFDA', 'align': 'center', 'valign': 'vcenter', 'border': 1}
    base_yld = {'bold': True, 'font_size': 18, 'font_color': '#2E7D32', 'align': 'center', 'valign': 'vcenter', 'border': 1}
    base_thin = {'align': 'center', 'valign': 'vcenter', 'border': 1}

    # [Fix 2] 좌표별 테두리 동적 생성기 (굵은 외곽선 대응)
    def get_fmt(base_props, top=1, bottom=1, left=1, right=1):
        props = base_props.copy()
        props.update({'top': top, 'bottom': bottom, 'left': left, 'right': right})
        return workbook.add_format(props)

    # 2. 컬럼 너비 및 여백 설정 (A열 마진 확보)
    ws.set_column('A:A', 3)
    ws.set_column('B:B', 15); ws.set_column('C:C', 22); ws.set_column('D:F', 10); ws.set_column('G:N', 11)

    # 3. 대시보드 영역 (B2:N6)
    # [Fix 3] 병합 셀의 우측 끝(N열) 굵은 테두리 유실 방지
    ws.merge_range('B2:F2', '📝 PRODUCTION SUMMARY', get_fmt(base_blue, top=2, left=2))
    ws.merge_range('G2:N2', '📈 CHANNEL STATISTICS', get_fmt(base_green, top=2, right=2))

    # Production Summary 작성 (B열 왼쪽 굵게)
    summary_items = [("Model Type", "MIC_MODEL_01"), ("Model P/N", "96575-XXXXX"), ("Prod. Date", "2026/02/05"), ("Quantity", "100 EA")]
    for i, (k, v) in enumerate(summary_items):
        r = 2 + i
        ws.write(r, 1, k, get_fmt(base_blue, left=2, bottom=2 if r==5 else 1))
        ws.write(r, 2, v, get_fmt(base_thin, bottom=2 if r==5 else 1))
    
    # PASS/FAIL/YIELD (6행 하단 굵은 구분선 적용)
    ws.write(2, 3, 'PASS', workbook.add_format(base_blue))
    ws.merge_range('E3:F3', 95, workbook.add_format(base_thin))
    ws.write(3, 3, 'FAIL', workbook.add_format(base_blue))
    ws.merge_range('E4:F4', 5, workbook.add_format(base_thin))
    ws.merge_range('D5:F6', "Yield: 95.0%", get_fmt(base_yld, bottom=2))

    # [Fix 4] 채널 수와 무관하게 N열 우측 테두리 유지 로직
    heads = ["Channel", "Pass", "Fail", "Yield", "Min", "Max", "Avg", "Stdev"]
    for i, h in enumerate(heads):
        col = 6 + i
        ws.write(2, col, h, get_fmt(base_green, right=2 if col==13 else 1))
    
    # 2개 채널만 있는 상황 가정 (나머지 행 빈칸 처리)
    for r_idx in range(4): # 3, 4, 5, 6행
        r = 3 + r_idx
        is_last = (r == 5)
        if r_idx < 2: # 채널 데이터 작성
            ws.write(r, 6, f"Ch {r_idx+1}", get_fmt(base_thin, bottom=2 if is_last else 1))
            for c_idx in range(1, 8):
                col = 6 + c_idx
                ws.write(r, col, "Data", get_fmt(base_thin, right=2 if col==13 else 1, bottom=2 if is_last else 1))
        else: # [Fix 5] 빈 행에서도 우측 굵은 테두리(N열)만 강제 드로잉
            for c_idx in range(8):
                col = 6 + c_idx
                ws.write_blank(r, col, "", workbook.add_format({'right': 2 if col==13 else 0, 'bottom': 2 if is_last else 0}))

    # [Fix 6] 그래프 영역(B7:N36) 내부 잔선 제거 기법
    # 안쪽 면 테두리를 0으로 설정하여 G열~M열 사이의 불필요한 실선 차단
    for r_frame in range(6, 36):
        # B열: 왼쪽만 굵게, 오른쪽(안쪽)은 0
        ws.write_blank(r_frame, 1, "", workbook.add_format({'left': 2, 'right': 0}))
        # N열: 오른쪽만 굵게, 왼쪽(안쪽)은 0
        ws.write_blank(r_frame, 13, "", workbook.add_format({'right': 2, 'left': 0}))

    # [Fix 7] 최하단 37행 마감 (중앙 잔선 없이 하단만 굵게)
    ws.write_blank(36, 1, "", workbook.add_format({'left': 2, 'bottom': 2, 'right': 0}))
    for c_bot in range(2, 13):
        ws.write_blank(36, c_bot, "", workbook.add_format({'bottom': 2, 'left': 0, 'right': 0, 'top': 0}))
    ws.write_blank(36, 13, "", workbook.add_format({'right': 2, 'bottom': 2, 'left': 0}))

    # [Fix 8] 이미지 미세조정 안내 (Pixel Offset)
    ws.write('B7', "← FR Chart (B7 Start, X_OFFSET=10, Y_OFFSET=10)")
    ws.write('H7', "← Cpk Chart (H7 Start, X_OFFSET=10, Y_OFFSET=10)")

    workbook.close()
    print(f"기록용 디버깅 파일 생성 완료: {output}")

if __name__ == "__main__":
    generate_recorded_layout()

기록용 디버깅 파일 생성 완료: Dashboard_Excel.xlsx


In [24]:
import xlsxwriter

def generate_sn_design_debug():
    output = "Defect_Table_Excel.xlsx"
    workbook = xlsxwriter.Workbook(output)
    ws = workbook.add_worksheet('🔍 결함상세')

    # 1. 서식 정의
    # 기본 스타일
    base_blue = {'bold': True, 'bg_color': '#DEEAF6', 'align': 'center', 'valign': 'vcenter', 'border': 1}
    base_thin = {'align': 'center', 'valign': 'vcenter', 'border': 1}
    # SN 행 전용 (연한 회색 배경)
    sn_bg = '#F2F2F2'
    
    # 위치별 서식 생성기
    def get_fmt(base_props, r, c, last_r):
        props = base_props.copy()
        # 외곽 굵은선 (Border 2)
        if r == 1: props['top'] = 2
        if r == last_r: props['bottom'] = 2
        if c == 1: props['left'] = 2
        if c == 13: props['right'] = 2
        if r == 5: props['bottom'] = 2 # 6행 구분선
        return workbook.add_format(props)

    # 2. 레이아웃 설정
    ws.set_column('A:A', 3)
    ws.set_column('B:B', 15); ws.set_column('C:C', 15); ws.set_column('D:D', 15) # SN 영역 너비
    ws.set_column('G:N', 11)

    # 3. 샘플 결함 데이터
    sample_defects = [
        {"sn": "96575GJ010/6A26010700002", 
         "table": [["Analog Mic", -15.2, -10.5, -9.8, "5.2%", "Defect"]]},
        {"sn": "96575GJ010/6A26010700004", 
         "table": [["Digital Ch1", -38.1, -37.2, -36.5, "1.2%", "Normal"]]}
    ]

    # 가변 높이 계산 (데이터 양에 따라 프레임 결정)
    last_row_idx = max(36, 9 + (len(sample_defects) * 5))

    # [기본 프레임 생략 - 이전과 동일한 로직으로 외곽선 그림]
    # (디버깅을 위해 생략하지만 실제 코드에는 포함됨)

    # 4. DETAILED FAILURE LOG 헤더 (B8:N8)
    fmt_log_header = workbook.add_format({**base_blue, 'left': 2, 'right': 2})
    ws.merge_range('B8:N8', '🔍 DETAILED FAILURE LOG', fmt_log_header)

    # 5. [핵심] 결함 데이터 및 SN 행 디자인 작성
    curr_r = 8 # 9행 시작
    for defect in sample_defects:
        # --- SN 행 디자인 (B, C, D열 색상 및 테두리 적용) ---
        # B열: 텍스트 기입 + 왼쪽 굵은선
        ws.write(curr_r, 1, defect["sn"], get_fmt({'bold':True, 'bg_color':sn_bg, 'border':1, 'align':'left'}, curr_r, 1, last_row_idx))
        # C열: 빈 셀 + 배경색 + 일반 테두리
        ws.write_blank(curr_r, 2, "", get_fmt({'bg_color':sn_bg, 'border':1}, curr_r, 2, last_row_idx))
        
        curr_r += 1
        
        # 데이터 테이블 헤더 (Channel, 200Hz, ...)
        headers = ["Channel", "200Hz", "1000Hz", "4000Hz", "THD", "Status"]
        for i, h in enumerate(headers):
            ws.write(curr_r, 1+i, h, get_fmt(base_blue, curr_r, 1+i, last_row_idx))
        curr_r += 1
        
        # 데이터 본문
        for row_data in defect["table"]:
            for i, val in enumerate(row_data):
                ws.write(curr_r, 1+i, val, get_fmt(base_thin, curr_r, 1+i, last_row_idx))
            curr_r += 1
        
        curr_r += 1 # 시료 간 공백

    # 6. 외곽 기둥 보정 (내부 격자 없이)
    for r in range(1, last_row_idx + 1):
        if r == 1 or r == last_row_idx or r == 5 or r == 7: continue
        # B열(1)과 N열(13)의 외곽선이 끊기지 않게 보강
        ws.write_blank(r, 1, "", workbook.add_format({'left': 2})) if r > 7 and not ws.table.get(r, {}).get(1) else None
        ws.write_blank(r, 13, "", workbook.add_format({'right': 2})) if not ws.table.get(r, {}).get(13) else None

    # 최하단 가로선
    for c in range(1, 14):
        if not ws.table.get(last_row_idx, {}).get(c):
            ws.write_blank(last_row_idx, c, "", workbook.add_format({'bottom': 2}))

    workbook.close()
    print(f"SN 디자인 개선 디버깅 완료: {output}")

if __name__ == "__main__":
    generate_sn_design_debug()

SN 디자인 개선 디버깅 완료: Defect_Table_Excel.xlsx
